# Inferring the growth rate using Gaussian process regression with Poisson link function

In [49]:
using Plots,Distributions,GaussianProcesses, Random,JLD2,Optim
plotlyjs()

Plots.PlotlyJSBackend()

In [50]:
#Case data for Kenya - this is scraped from the background html generating this page https://www.worldometers.info/coronavirus/country/kenya/

case_dates = ["Feb 15","Feb 16","Feb 17","Feb 18","Feb 19","Feb 20","Feb 21","Feb 22","Feb 23","Feb 24","Feb 25","Feb 26","Feb 27",
                "Feb 28","Feb 29","Mar 01","Mar 02","Mar 03","Mar 04","Mar 05","Mar 06","Mar 07","Mar 08","Mar 09","Mar 10","Mar 11",
                "Mar 12","Mar 13","Mar 14","Mar 15","Mar 16","Mar 17","Mar 18","Mar 19","Mar 20","Mar 21","Mar 22","Mar 23","Mar 24",
                "Mar 25","Mar 26","Mar 27","Mar 28","Mar 29","Mar 30","Mar 31","Apr 01","Apr 02","Apr 03","Apr 04","Apr 05","Apr 06",
                "Apr 07","Apr 08","Apr 09","Apr 10","Apr 11","Apr 12","Apr 13","Apr 14","Apr 15","Apr 16","Apr 17","Apr 18","Apr 19",
                "Apr 20","Apr 21","Apr 22","Apr 23","Apr 24","Apr 25","Apr 26","Apr 27","Apr 28","Apr 29","Apr 30","May 01","May 02",
                "May 03","May 04","May 05","May 06","May 07","May 08","May 09","May 10","May 11","May 12","May 13","May 14","May 15",
                "May 16","May 17","May 18","May 19","May 20","May 21","May 22","May 23","May 24","May 25","May 26","May 27","May 28",
                "May 29","May 30","May 31","Jun 01","Jun 02","Jun 03","Jun 04"]  
f = findfirst(case_dates .== "Mar 13")
case_dates = case_dates[f:end]

#Shorten data to be from 13th March

incidence = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,3,0,0,0,8,
                    1,9,3,3,0,7,4,8,9,22,29,12,4,16,16,14,7,5,5,2,6,11,8,9,9,12,16,8,11,15,7,17,16,7,12,8,11,10,12,15,24,30,25,45,47,25,14,28,23,28,15,22,21,23,49,57,25,51,66,
                    80,52,31,22,72,62,123,147,127,143,74,59,72,123,124] 
incidence = incidence[f:end]

scatter(incidence,lab = "",
    xticks = (1:7:length(case_dates),case_dates[1:7:end]),
    ylabel = "Daily new cases reported",
    title = "Reported incidence (Mar 13th - Jun 4th)")

<!DOCTYPE html>
 
 
 Plots.jl

In [71]:
T = Float64.(collect(1:length(incidence)))
#GP set-up
k = Matern(3/2,log(7.),log(1.))   # Matern 3/2 kernel
l = PoisLik()             # Poisson likelihood
gpmc = GP(T, incidence, MeanZero(), k, l)


7.0

In [69]:
optimize!(gpmc) # Optimize the kernel parameters in a box with lower bounds [-1, -1] and upper bounds [1, 1]
# plot(gp)

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer: [-8.55e-02, -7.45e-02, 4.98e-02,  ...]
    Minimum:   2.571721e+02

 * Found with
    Algorithm:     L-BFGS
    Initial Point: [0.00e+00, 0.00e+00, 0.00e+00,  ...]

 * Convergence measures
    |x - x'|               = 9.50e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 2.08e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.12e-05 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 4.37e-08 ≰ 0.0e+00
    |g(x)|                 = 1.84e+00 ≰ 1.0e-08

 * Work counters
    Seconds run:   133  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    2899
    ∇f(x) calls:   2899


In [28]:
n = 20
# X = collect(range(-3,stop=3,length=n));
X = Float64.(collect(1:length(incidence)))
f = 2*cos.(2*X);
Y = [rand(Poisson(exp.(f[i]))) for i in 1:n];

In [35]:
#GP set-up
k = Matern(3/2,0.0,0.0)   # Matern 3/2 kernel
l = PoisLik()             # Poisson likelihood
gpmc = GP(X, incidence, MeanZero(), k,l)


GP Approximate object:
  Dim = 1
  Number of observations = 84
  Mean function:
    Type: MeanZero, Params: Float64[]
  Kernel:
    Type: Mat32Iso{Float64}, Params: [0.0, 0.0]
  Likelihood:
    Type: PoisLik, Params: Any[]
  Input observations = 
[1.0 2.0 … 83.0 84.0]
  Output observations = [0, 0, 2, 0, 1, 3, 0, 0, 0, 8  …  62, 123, 147, 127, 143, 74, 59, 72, 123, 124]
  Log-posterior = -7212.153

In [36]:
set_priors!(gpmc.kernel,[Normal(-2.0,4.0),Normal(-2.0,4.0)])
@time samples = mcmc(gpmc; nIter=10000);

@save "GP_mcmc_samples.jld2" s


Number of iterations = 10000, Thinning = 1, Burn-in = 1 
Step size = 0.100000, Average number of leapfrog steps = 9.961400 
Number of function calls: 99611
Acceptance rate: 0.000000 
2518.122049 seconds (1.07 G allocations: 112.093 GiB, 0.67% gc time)


In [38]:
@save "GP_mcmc_samples.jld2" samples

In [78]:
x = 2π * rand(10)
y = sin.(x) + 0.1*rand(10)

# Set-up mean and kernel
se = SE(0.0, 0.0)
m = MeanZero()

# Construct and plot GP
gp = GP(x,y,m,se)
plot(gp)

┌ Warning: Attribute alias `color` detected in the user recipe defined for the signature (::GPE{LinearAlgebra.Adjoint{Float64,Array{Float64,1}},Array{Float64,1},MeanZero,SEIso{Float64},GaussianProcesses.FullCovariance,GaussianProcesses.IsotropicData{Array{Float64,2}},PDMats.PDMat{Float64,Array{Float64,2}},GaussianProcesses.Scalar{Float64}}). To ensure expected behavior it is recommended to use the default attribute `seriescolor`.
└ @ Plots /Users/Sam/.julia/packages/Plots/JKY3H/src/pipeline.jl:15


<!DOCTYPE html>
 
 
 Plots.jl